<a href="https://colab.research.google.com/github/taejjing/cloud_tpu_example/blob/master/TPU_init_1_14_make_datasets_3rd_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading Tensorflow & check the version 1.14

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()
# AUTOTUNE = tf.data.experimental.AUTOTUNE

print(tf.__version__)

1.14.0


## Google auth login

If you wanna use Colab TPU, you must access your GCS bucket.
so,  you should run the below code to get permession to access your own GCS.

In [0]:
from google.colab import auth
auth.authenticate_user()

## Kaggle init & Download data
you should upload that file to this vm for using kaggle api to download kaggle datasets.
My kaggle key is stored on my own bucket.

In [0]:
DATA_PATH = '/content'

In [0]:
burketName = "Your Bucket Name" #@param {type:"string"}

! gsutil cp {burketName}/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

Copying gs://tj-kaggle/kaggle.json...
/ [1 files][   63.0 B/   63.0 B]                                                
Operation completed over 1 objects/63.0 B.                                       


In [0]:
! kaggle competitions download -c 2019-3rd-ml-month-with-kakr -p {DATA_PATH}

  0% 0.00/108k [00:00<?, ?B/s]
100% 108k/108k [00:00<00:00, 33.8MB/s]
  0% 0.00/184k [00:00<?, ?B/s]
100% 184k/184k [00:00<00:00, 54.7MB/s]
100% 699M/700M [00:12<00:00, 70.6MB/s]
100% 700M/700M [00:13<00:00, 56.3MB/s]
  0% 0.00/343k [00:00<?, ?B/s]
100% 343k/343k [00:00<00:00, 110MB/s]
 99% 1.13G/1.14G [00:19<00:00, 87.1MB/s]
100% 1.14G/1.14G [00:19<00:00, 62.2MB/s]
  0% 0.00/6.19k [00:00<?, ?B/s]
100% 6.19k/6.19k [00:00<00:00, 5.39MB/s]


In [0]:
%%time
! unzip -q train.zip -d train
! unzip -q test.zip -d test

CPU times: user 152 ms, sys: 31 ms, total: 183 ms
Wall time: 22.6 s


## Data Load

In [0]:
import os
import pandas as pd

In [0]:
# 이미지 폴더 경로
TRAIN_IMG_PATH = os.path.join(DATA_PATH, 'train')
TEST_IMG_PATH = os.path.join(DATA_PATH, 'test')

# CSV 파일 경로
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
df_class = pd.read_csv(os.path.join(DATA_PATH, 'class.csv'))

In [0]:
train_size = df_train.shape[0]

In [0]:
df_train['img_file'] = df_train['img_file'].map(lambda x : "/content/train/" + x)
all_image_paths = df_train['img_file'].tolist()
all_image_labels  = df_train['class'].tolist()

In [0]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [0]:
# Create a dictionary with features that may be relevant.
def image_example(image_string, label):
  
  image_shape = tf.image.decode_jpeg(image_string).shape

  feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }
  return tf.train.Example(features=tf.train.Features(feature=feature))

In [0]:
with tf.python_io.TFRecordWriter('images.tfrecords') as writer:
  for i in range(train_size) :
    filename = all_image_paths[i]
    label = all_image_labels[i]
    
    image_string = open(filename, 'rb').read()
    
    tf_example = image_example(image_string, label)
    
    writer.write(tf_example.SerializeToString())

## Upload tfrecord file to your own GCS

In [0]:
!gsutil -m cp /content/images.tfrecords  gs://{burketName}/myData/ 

Copying file:///content/images.tfrecords [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

|
Operation completed over 1 objects/1.2 GiB.                                      
